In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            # print(f"Scenario {scenario_num} not found in the HDF5 file.")
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "../synthetic_data/RCT_0_5.h5",
    "../synthetic_data/RCT_0_05.h5",
    "../synthetic_data/e_X.h5",
    "../synthetic_data/e_X_U.h5",
    "../synthetic_data/e_X_no_overlap.h5",
    "../synthetic_data/e_X_info_censor.h5",
    "../synthetic_data/e_X_U_info_censor.h5",
    "../synthetic_data/e_X_no_overlap_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            print(f"Error loading {path} scenario {scenario}: {e}")
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [5]:
experiment_repeat_setups = pd.read_csv("../synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# EXPERIMENT CONSTANTS

In [6]:
num_repeats_to_include = 2  # max 10
num_training_data_points = 200 # max 45000
test_size = 5000
survival_model = 'DeepHit' # 
meta_learner_type = 't_learner_survival'
# load_imputed_values = True
# imputed_times_path = f"synthetic_data/imputed_times_lookup.pkl"

In [7]:
output_pickle_path = f"results/{meta_learner_type}_{survival_model}_num_repeats_{num_repeats_to_include}_train_size_{num_training_data_points}.pkl"

# Run Experiments

In [8]:
# from models_causal_impute.meta_learners import TLearner, SLearner, XLearner
# from models_causal_impute.survival_eval_impute import SurvivalEvalImputer
import sys
import os

# Add the parent directory of "notebooks" to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from models_causal_survival_meta.meta_learners_survival import TLearnerSurvival, SLearnerSurvival, MatchingLearnerSurvival
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [9]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list, num_training_data_points=5000, test_size=5000):
    split_results = {}

    for rand_idx in random_idx_col_list:
        random_idx = experiment_repeat_setups[rand_idx].values
        test_ids = random_idx[-test_size:]
        train_ids = random_idx[:min(num_training_data_points, len(random_idx) - test_size)]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        cate_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[rand_idx] = (X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true)

    return split_results

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

experiment_repeat_setups = pd.read_csv("synthetic_data/idx_split.csv").set_index("idx")
random_idx_col_list = experiment_repeat_setups.columns.to_list()[:num_repeats_to_include]

output_pickle_path = f"results/{meta_learner_type}_{survival_model}_{'mean'}_repeats_{num_repeats_to_include}_train_{num_training_data_points}.pkl"
print("Output results path:", output_pickle_path)

# Define base survival models to use
base_model = survival_model
results_dict = {}

# Define hyperparameter grids for each model
hyperparameter_grids = {
    'RandomSurvivalForest': {
        'n_estimators': [50, 100],
        'min_samples_split': [5, 10],
        'min_samples_leaf': [3, 5]
    },
    'DeepSurv': {
        'num_nodes': [32, 64],
        'dropout': [0.1, 0.2],
        'lr': [0.01, 0.001],
        'epochs': [100, 500]
    },
    'DeepHit': {
        'num_nodes': [32, 64],
        'dropout': [0.1, 0.2],
        'lr': [0.01, 0.001],
        'epochs': [100, 500]
    }
}

counter = 0

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):
    results_dict[setup_name] = {}
    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios"):

        # counter += 1
        # if counter < 5:
        #     continue

        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list, num_training_data_points)
        results_dict[setup_name][scenario_key] = {}

        start_time = time.time()

        for rand_idx in random_idx_col_list:
            X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true = split_dict[rand_idx]

            max_time = Y_train[:, 0].max()
            
            # Initialize the appropriate meta-learner
            if meta_learner_type == "t_learner_survival":
                learner = TLearnerSurvival(
                    base_model_name=base_model,
                    base_model_grid=hyperparameter_grids,
                    metric="mean",
                    max_time=max_time
                )
            elif meta_learner_type == "s_learner_survival":
                learner = SLearnerSurvival(
                    base_model_name=base_model,
                    base_model_grid=hyperparameter_grids,
                    metric="mean",
                    max_time=max_time
                )
            elif meta_learner_type == "matching_survival":
                learner = MatchingLearnerSurvival(
                    base_model_name=base_model,
                    base_model_grid=hyperparameter_grids,
                    metric="mean",
                    num_matches=5,
                    max_time=max_time
                )

            # Fit the learner
            learner.fit(X_train, W_train, Y_train)
            
            # Evaluate base survival models on test data
            base_model_eval = learner.evaluate_test(X_test, Y_test, W_test)
            
            # Evaluate causal effect predictions
            mse_test, cate_test_pred, ate_test_pred = learner.evaluate(X_test, cate_test_true, W_test)

            results_dict[setup_name][scenario_key][rand_idx] = {
                "cate_true": cate_test_true,
                "cate_pred": cate_test_pred,
                "ate_true": cate_test_true.mean(),
                "ate_pred": ate_test_pred,
                "cate_mse": mse_test,
                "ate_bias": ate_test_pred - cate_test_true.mean(),
                "base_model_eval": base_model_eval  # Store base model evaluation results
            }

        end_time = time.time()
        avg = results_dict[setup_name][scenario_key]
        results_dict[setup_name][scenario_key]["average"] = {
            "mean_cate_mse": np.mean([avg[i]["cate_mse"] for i in random_idx_col_list]),
            "std_cate_mse": np.std([avg[i]["cate_mse"] for i in random_idx_col_list]),
            "mean_ate_pred": np.mean([avg[i]["ate_pred"] for i in random_idx_col_list]),
            "std_ate_pred": np.std([avg[i]["ate_pred"] for i in random_idx_col_list]),
            "mean_ate_true": np.mean([avg[i]["ate_true"] for i in random_idx_col_list]),
            "std_ate_true": np.std([avg[i]["ate_true"] for i in random_idx_col_list]),
            "mean_ate_bias": np.mean([avg[i]["ate_bias"] for i in random_idx_col_list]),
            "std_ate_bias": np.std([avg[i]["ate_bias"] for i in random_idx_col_list]),
            "runtime": (end_time - start_time) / len(random_idx_col_list),
            "base_model_eval" : {
                                    base_model_k: {
                                        f"{stat}_{metric_j}": func([
                                            avg[i]['base_model_eval'][base_model_k][metric_j] for i in random_idx_col_list
                                        ])
                                        for metric_j in metric_j_dict
                                        for stat, func in zip(['mean', 'std'], [np.nanmean, np.nanstd])
                                    }
                                    for base_model_k, metric_j_dict in avg[random_idx_col_list[0]]['base_model_eval'].items()
                                }
            }
    
    break

Output results path: results/t_learner_survival_DeepHit_mean_repeats_2_train_200.pkl


Experiment Setups:   0%|          | 0/8 [00:00<?, ?it/s]

Error calculating integrated brier score: all times must be within follow-up time of test data: [0.009524181732724112; 1.2595473527908325[


Error calculating integrated brier score: all times must be within follow-up time of test data: [1.0; 11.0[


Error calculating integrated brier score: all times must be within follow-up time of test data: [1.0; 13.0[


Error calculating integrated brier score: all times must be within follow-up time of test data: [0.0; 8.0[


Experiment Setups:   0%|          | 0/8 [01:54<?, ?it/s]
